# Comprehensive Vertex AI Tutorial For Beginners

## Introduction

Even though most people associate search engines with Google, the company has deep roots in the data science industry. It consistently delivers state-of-the-art products and solutions aimed at getting the most benefit from data. One of their products, Vertex AI, was released in 2021 to simplify the machine learning process at enterprise scale.

In this tutorial, we will learn how to get started with Google's Vertex AI platform and how to use it to cover a wider range of tasks of the ML life cycle. We will leave with a deployed model we can send requests to generate predictions for a classification task. 

Let's get started.

## Why learn Vertex AI?

## What are Google Cloud Services?

## Setting up your Google Cloud Console For Vertex AI

1. Go to https://console.cloud.google.com/
2. Click on the top left bar to create your new project - CREATE A GIF
3. Go to https://console.cloud.google.com/billing to create a billing account
4. Add billing account to your project
5. Enable APIs related to Vertex AI
6. Go to vertex AI solution 

## Uploading a dataset in Vertex AI

Show GIFs for uploading a dataset to Vertex AI

## Training and deploying AutoML models in Vertex AI

## Training and deploying custom models in Vertex AI

## Conclusion